# Classification using only Behavioral APIs

## Read in DataFrame

In [ ]:
import pandas as pd
import ast
import numpy as np

# Load the labelled behavioral biometric dataset into a DataFrame
df = pd.read_csv('/home/a8tariq/safeVM/latest_relabeled.csv')
df

## Remove Duplicates and Preprocess

In [ ]:
# Filter out rows where 'script_url' contains 'datadome'
df = df[~df['script_url'].str.contains('datadome', case=False, na=False)]
print(f"Dimensions of the dataset after dropping datadome: {df.shape}")
print("True samples BEFORE dropping duplicates: ", len(df[df["is_behavioral_biometric"] == 't']))

# Function to parse json object arrays and convert them to sorted sets for comparison
def parse_and_sort(value):
    if pd.isna(value):
        return np.nan  # Return NaN as is
    if isinstance(value, str):
        # If it's a string, evaluate and then sort and convert to a tuple
        try:
            return tuple(sorted(ast.literal_eval(value)))
        except (ValueError, SyntaxError):
            return value  # Return original if it can't be parsed
    elif isinstance(value, (set, list)):
        # If it's already a set or list, just sort and convert to a tuple
        return tuple(sorted(value))
    return value  # If it's something else, return as is

# Apply the function to the relevant columns using .loc[] to avoid SettingWithCopyWarning
for col in ["max_aggregated_apis", "aggregated_behavioral_apis", "aggregated_fingerprinting_apis", 
            "fingerprinting_source_apis", "behavioral_source_apis"]:
    df.loc[:, col] = df[col].apply(parse_and_sort)

# Remove duplicates only based on the specified columns
df_cleaned = df.drop_duplicates(subset=[
    "max_api_aggregation_score", "behavioral_api_agg_count", "fp_api_agg_count", 
    "max_behavioral_api_aggregation_score", "max_fingerprinting_api_aggregation_score", 
    "behavioral_source_api_count", "fingerprinting_source_api_count", 
    "graph_construction_failure", "max_aggregated_apis", "aggregated_behavioral_apis", 
    "aggregated_fingerprinting_apis", "fingerprinting_source_apis", 
    "behavioral_source_apis"
])

print(f"Dimensions of the dataset after dropping duplicates: {df_cleaned.shape}")
print("True samples after dropping duplicates: ", len(df_cleaned[df_cleaned["is_behavioral_biometric"] == 't']))
# Remove unnecessary features
df_cleaned = df_cleaned.drop(columns=['script_url', 'script_id', 'code', 'attached_listeners'], axis=1)
df_cleaned.head()

print(f"Dimensions of the dataset : {df_cleaned.shape}")


## Create Simple DataFrame with Behavioral APIs

In [ ]:
# Create a new dataframe with simple features 
simple_df = df_cleaned[['behavioral_source_apis', 'behavioral_source_api_count', 'is_behavioral_biometric']].copy()

# Parse the string array into a Python list
# simple_df['behavioral_source_apis'] = simple_df['behavioral_source_apis'].apply(ast.literal_eval)

# 1. Extract the string before the period (e.g., 'MouseEvent', 'KeyboardEvent', etc.)
simple_df['behavioral_source_apis'] = simple_df['behavioral_source_apis'].apply(lambda x: [event.split('.')[0] for event in x])

# 2. Flatten the list into individual event types and count the occurrences
# We will first explode the list of event types and then use pivot_table to count occurrences
df_exploded = simple_df.explode('behavioral_source_apis')

# 3. Create new columns for each event type and count how many times they appear per row
df_count = df_exploded.pivot_table(index=df_exploded.index, columns='behavioral_source_apis', aggfunc='size', fill_value=0)

# 4. Join the new columns with the original DataFrame
df_final = simple_df.join(df_count)

# 5. Optional: Drop the original 'events' column if not needed
df_final.drop(columns='behavioral_source_apis', inplace=True)
df_final.drop(columns='behavioral_source_api_count', inplace=True)
df_final


## Unbalanced Prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df_final.drop('is_behavioral_biometric', axis=1)
Y = df_final['is_behavioral_biometric']
# Split the data into training and testing sets (90% train, 10% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=45)

# Initialize and train a Decision Tree Classifier
clf = RandomForestClassifier( random_state=45)
clf.fit(X_train, y_train)

# Make predictions on the test data
train_pred = clf.predict(X_test)

# Make predictions on the training data
y_train_pred = clf.predict(X_train)

# Make predictions on the test data
y_test_pred = clf.predict(X_test)

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

## Cross Validation Results (Unbalanced)

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the balanced training set
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Average cross-validation score: {scores.mean()}")

## Class Weighting Prediction

In [ ]:
# Class balance property should handle under-represented part of dataset
clf = RandomForestClassifier(class_weight='balanced', random_state=45)

# Step 3: Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
train_pred = clf.predict(X_test)

# Make predictions on the training data
y_train_pred = clf.predict(X_train)

# Make predictions on the test data
y_test_pred = clf.predict(X_test)

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))



## Cross Validation Results (Class Weighting)

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the balanced training set
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Average cross-validation score: {scores.mean()}")


## Oversampled Prediction

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Step 2: Balance the training set using oversampling (or undersampling, SMOTE, etc.)
ros = RandomOverSampler(random_state=45)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


# Train a Decision Tree classifier on the balanced dataset
clf = RandomForestClassifier(random_state=45)
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
train_pred = clf.predict(X_test)

# Make predictions on the training data
y_train_pred = clf.predict(X_train)

# Make predictions on the test data
y_test_pred = clf.predict(X_test)

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))


## Cross Validation Results (Oversampled)

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the balanced training set
scores = cross_val_score(clf, X_train_resampled, y_train_resampled, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Average cross-validation score: {scores.mean()}")


## Undersampled Prediction

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Step 2: Balance the training set using oversampling (or undersampling, SMOTE, etc.)
ros = RandomUnderSampler(random_tate=45)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


# Train a Decision Tree classifier on the balanced dataset
clf = RandomForestClassifier(random_state=45)
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions and evaluate
y_pred = clf.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

## Cross Validation Results (Undersampled)

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the balanced training set
scores = cross_val_score(clf, X_train_resampled, y_train_resampled, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Average cross-validation score: {scores.mean()}")


## ROC-AUC Results

In [ ]:
from sklearn.metrics import roc_auc_score

# Get probability predictions for ROC-AUC score
y_pred_proba = clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc:.2f}")
